<a href="https://colab.research.google.com/github/luzyi0/2025-Vision_Computer_TI2B/blob/main/PRAKTIKUM05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persiapan Environment

In [ ]:
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

CUDA Available: False
CUDA Device: No GPU


In [ ]:
# Install PyTorch dan torchvision
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install library pendukung
!pip install opencv-python matplotlib pillow numpy
!pip install pycocotools  # Untuk evaluasi dengan COCO metrics

# Install library untuk selective search (R-CNN dan Fast R-CNN)
!pip install selectivesearch

# Verify installation
import torch
import torchvision
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print(f"PyTorch Version: {torch.__version__}")
print(f"Torchvision Version: {torchvision.__version__}")
print(f"OpenCV Version: {cv2.__version__}")

Looking in indexes: https://download.pytorch.org/whl/cu118
PyTorch Version: 2.8.0+cu126
Torchvision Version: 0.23.0+cu126
OpenCV Version: 4.12.0


In [ ]:
# Download dan extract dataset
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
!unzip -q PennFudanPed.zip
!ls PennFudanPed/

--2025-11-21 01:28:27--  https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
Resolving www.cis.upenn.edu (www.cis.upenn.edu)... 158.130.69.163, 2607:f470:8:64:5ea5::d
Connecting to www.cis.upenn.edu (www.cis.upenn.edu)|158.130.69.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53723336 (51M) [application/zip]
Saving to: ‘PennFudanPed.zip’

PennFudanPed.zip    100%[===================>]  51.23M  64.3MB/s    in 0.8s    

2025-11-21 01:28:28 (64.3 MB/s) - ‘PennFudanPed.zip’ saved [53723336/53723336]

added-object-list.txt  Annotation  PedMasks  PNGImages	readme.txt


In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import time

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


# Praktikum

## Praktikum 1: Implementasi R-CNN

### Langkah 1.1: Implementasi Sselective Search

In [ ]:
import slectivesearch

def generate_region_proposals(image_path, scale=500, sigma=0.9, min_size=10):
  """
  Generate region proposals using Selective Search

  Args:
        image_path: Path to input image
        scale: Larger sclae -> fewer regions
        sigma: Width of Gaussian kernel for preprocessing